In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

c:\Users\yona\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [5]:
def properties_info(html):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
    response = requests.get(html, headers=headers)
    if response is None or not response.status_code == 200:
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    my_dict = {}
    city = html.split('/for-sale/')[1].split('-ישראל')[0]
    property_class_elements = soup.find_all('div', attrs={'data-auto': 'property-class'})
    property_class = [element.text.strip() for element in property_class_elements]
    property_area_elements = soup.find_all('div', attrs={'data-auto': 'property-size'})
    property_area = [element.text.strip() for element in property_area_elements]
    property_address_elements = soup.find_all('div', class_='css-v4h09j e1sx3tzs16', attrs={'data-auto': 'property-address'})
    neighborhoods = [element.text.strip().split(',')[-1].strip() for element in property_address_elements]
    property_address_elements = soup.findAll('div', attrs={'data-auto': 'property-address'})
    property_street = []
    property_numbers = []
    for element in property_address_elements:
        address_parts = element.text.strip().split(',')
        if len(address_parts) > 1:
            street = address_parts[0].strip()
            number = re.findall(r'\b\d+\b', element.text.strip())
            if number:
                number = ' '.join(number)
            else:
                number = None
        else:
            street = element.text.strip()
            number = None

        property_street.append(street)
        property_numbers.append(number)
    property_price_elements = soup.find_all('div', {'data-auto': 'property-price'})
    property_prices = []
    for element in property_price_elements:
        price_text = element.text.strip().split('₪')[1]
        price_numbers = re.findall(r'\d', price_text)
        if price_numbers:
            property_prices.append(int(''.join(price_numbers)))
        else:
            property_prices.append(None) 

    max_length = max(len(lst) for lst in [property_class, property_area, property_street, property_numbers, neighborhoods, property_prices])
    property_class += [None] * (max_length - len(property_class))
    property_area += [None] * (max_length - len(property_area))
    property_street += [None] * (max_length - len(property_street))
    property_numbers += [None] * (max_length - len(property_numbers))
    neighborhoods += [None] * (max_length - len(neighborhoods))
    property_prices += [None] * (max_length - len(property_prices))
    my_dict['City'] = city
    my_dict['Type'] = property_class
    my_dict['Area'] = property_area
    my_dict['Street'] = [re.sub(r'\b\d+\b', '', street).strip() if street else None for street in property_street]
    my_dict['number_in_the_street'] = property_numbers
    my_dict['city_area'] = neighborhoods
    my_dict['price'] = property_prices

    df = pd.DataFrame.from_dict(my_dict)
    return df,my_dict,city

In [6]:
df,my_dict,city = properties_info('https://www.madlan.co.il/for-sale/נתניה-ישראל')
df

,City,Type,Area,Street,number_in_the_street,city_area,price
0,נתניה,דירה,"133 מ""ר",הגולן,11,קרית השרון,3250000
1,נתניה,דירה,"130 מ""ר",הטורים,3,בן ציון,2980000
2,נתניה,דירה,"75 מ""ר",מרכז העיר דרום,None,מרכז העיר דרום,1900000
3,נתניה,פנטהאוז,"145 מ""ר",צרפת,None,פרדס הגדוד,3850000
4,נתניה,בית פרטי,"280 מ""ר",השושנה,5,עין התכלת,7200000
5,נתניה,דירה,"111 מ""ר",בארי,77,מרכז העיר צפון,2300000
6,נתניה,דירה,"135 מ""ר",רזניק דוד,None,נוף גלים,3090000
7,נתניה,דירה,"137 מ""ר",דגניה,None,קרית השרון,3190000
8,נתניה,דירה,"128 מ""ר",גבע,4,מרכז העיר דרום,2750000
9,נתניה,אחר,"50 מ""ר",פתח תקוה,None,מרכז העיר צפון,1100000
